# **<span style="color:red;text-decoration:underline">Let's solve some sudokus!</span>**
In this notebook we will try to solve some sudokus using neural networks. As you will see that a neural net is not very suited for this use case so we would look at some of the most optimised approaches in the end.

<t>This tutorial is inspired by [this](https://towardsdatascience.com/solving-sudoku-with-convolution-neural-network-keras-655ba4be3b11) awesome medium article. This tutorial is beginner friendly so anyone with a basic understanding of python and neural nets(convnets) can proceed.

## <span style="color:green;text-decoration:underline">What is a sudoku?</span>
Sudoku is a logic-based, combinatorial number-placement puzzle. The objective is to fill a 9×9 grid with digits so that each column, each row, and each of the nine 3×3 subgrids that compose the grid contain all of the digits from 1 to 9.
<t>The modern Sudoku was most likely designed anonymously by Howard Garns, a 74-year-old retired architect and freelance puzzle constructor from Connersville, Indiana, and first published in 1979 by Dell Magazines as Number Place (the earliest known examples of modern Sudoku).
<t>
![img](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e0/Sudoku_Puzzle_by_L2G-20050714_standardized_layout.svg/300px-Sudoku_Puzzle_by_L2G-20050714_standardized_layout.svg.png)
    
### Rules
1. Each row, column, and subgrid should contain each number (1 to 9) exactly once.
2. The sum of all numbers in any subgrid, row, or column must be equal to 45.


## <span style="color:green;text-decoration:underline">Exploratory Data Analysis?</span>
Let's understand the data we are working with.
We are using the [9 Million Sudoku Puzzles and Solutions](https://www.kaggle.com/rohanrao/sudoku) dataset. 
<br>You can also combine the [1 million Sudoku games](https://www.kaggle.com/bryanpark/sudoku) dataset to have more data although 9M is already a lot.

In [ ]:
import numpy as np
import pandas as pd
import keras
import keras.backend as K
from keras.optimizers import Adam
from keras.models import Sequential
from keras.utils import Sequence
from keras.layers import *
import matplotlib.pyplot as plt

In [ ]:
path = "../input/sudoku/"
data = pd.read_csv(path+"sudoku.csv")
try:
    data = pd.DataFrame({"quizzes":data["puzzle"],"solutions":data["solution"]})
except:
    pass
data.head()

In [ ]:
data.info()

Quizes and Solutions are in the form of strings of length 81. '0' in the quizzes are for blank spaces.
<br>Let's see the quizes and solutions more semantically.

In [ ]:
print("Quiz:\n",np.array(list(map(int,list(data['quizzes'][0])))).reshape(9,9))
print("Solution:\n",np.array(list(map(int,list(data['solutions'][0])))).reshape(9,9))

## <span style="color:green;text-decoration:underline">Deep Learning Approach</span>
### Let's create a basic model
We know that semantics are important for solving a sudoku and convnets are ideal in preserving semantics and extracting semantic features so we will be creating a [convnet](https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53).

* Our input are batches of arrays of shape (9,9).
* Output is of shape (81,1) because we are using [sparse_categorical_crossentropy](https://stackoverflow.com/questions/58565394/what-is-the-difference-between-sparse-categorical-crossentropy-and-categorical-c) loss function which does not require passing one-hot vectors as output. 
* We are using a batch size of 640 as the data is very large in number and can easily fit in a decent GPU.
* Training and validation split is of 95% to 5% as 5% of 9M is also very large(4,50,000).
* We will be using reducing lr approach to finetune our model and model checkpointing to save the best model and avoid overfitting.

In [ ]:
#Utility Functions
class DataGenerator(Sequence):
    def __init__(self, df,batch_size = 16,subset = "train",shuffle = False, info={}):
        super().__init__()
        self.df = df
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.subset = subset
        self.info = info
        
        self.data_path = path
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.df)/self.batch_size))
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle==True:
            np.random.shuffle(self.indexes)
            
    def __getitem__(self,index):
        X = np.empty((self.batch_size, 9,9,1))
        y = np.empty((self.batch_size,81,1))
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        for i,f in enumerate(self.df['quizzes'].iloc[indexes]):
            self.info[index*self.batch_size+i]=f
            X[i,] = (np.array(list(map(int,list(f)))).reshape((9,9,1))/9)-0.5
        if self.subset == 'train': 
            for i,f in enumerate(self.df['solutions'].iloc[indexes]):
                self.info[index*self.batch_size+i]=f
                y[i,] = np.array(list(map(int,list(f)))).reshape((81,1)) - 1
        if self.subset == 'train': return X, y
        else: return X

In [ ]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

model.add(Flatten())
model.add(Dense(81*9))
model.add(Reshape((-1, 9)))
model.add(Activation('softmax'))

adam = keras.optimizers.adam(lr=.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
model.summary()

### Data Generators
We will be creating training and test data generator. Lets use 95% data for training and 5% data for validation as 5% of 9Million is still very large for validation purposes. 

In [ ]:
train_idx = int(len(data)*0.95)
data = data.sample(frac=1).reset_index(drop=True)
training_generator = DataGenerator(data.iloc[:train_idx], subset = "train", batch_size=640)
validation_generator = DataGenerator(data.iloc[train_idx:], subset = "train",  batch_size=640)

In [ ]:
training_generator.__getitem__(4)[0].shape

### Callbacks
Callbacks help in monitoring model training on-the-go. They can be used to stop training, saving best weights, reducing learning rate if validation accuracy is not improving.
<br>
In our case we are using two callbacks.
1. First one is **ModelCheckpoint** which saves the weights as soon as the validation accuracy imporves from the previous best.
2. Second one is **ReduceLROnPlateau** which reduces the learning rate if the validation loss doesnot improve after a set no of epochs called patience. In our case the patience is of 3 and the lowest it can go is 1e-6 after which it does not reduce LR.

In [ ]:
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
filepath1="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
filepath2 = "best_weights.hdf5"
checkpoint1 = ModelCheckpoint(filepath1, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
checkpoint2 = ModelCheckpoint(filepath2, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=3,
    verbose=1,
    min_lr=1e-6
)
callbacks_list = [checkpoint1,checkpoint2,reduce_lr]

In [ ]:
history = model.fit_generator(training_generator, validation_data = validation_generator, epochs = 1, verbose=1,callbacks=callbacks_list )

NOTE: For best results train for 5 to 6 epochs. I've tried training upto 30 epochs and the accuracy doesn't improve above 5 epochs.

In [ ]:
model.load_weights('best_weights.hdf5')

## <span style="color:green;text-decoration:underline">Solving Real Sudokus!!?</span>
Here we use a more human approach to solving the sudoku that is to fill one number at a time. This means that we pass the sudoku through the neural net once and fill the number that it is most sure of, than again pass it and fill another number till all the numbers are filled. This helps the neural net to gain context from the previously filled digits like a human player.

In [ ]:
def norm(a):
    return (a/9)-.5

def denorm(a):
    return (a+.5)*9

def inference_sudoku(sample):
    
    '''
        This function solve the sudoku by filling blank positions one by one.
    '''
    
    feat = sample
    
    while(1):
    
        out = model.predict(feat.reshape((1,9,9,1)))  
        out = out.squeeze()

        pred = np.argmax(out, axis=1).reshape((9,9))+1 
        prob = np.around(np.max(out, axis=1).reshape((9,9)), 2) 
        
        feat = denorm(feat).reshape((9,9))
        mask = (feat==0)
     
        if(mask.sum()==0):
            break
            
        prob_new = prob*mask
    
        ind = np.argmax(prob_new)
        x, y = (ind//9), (ind%9)

        val = pred[x][y]
        feat[x][y] = val
        feat = norm(feat)
    
    return pred

def test_accuracy(feats, labels):
    
    correct = 0
    
    for i,feat in enumerate(feats):
        
        pred = inference_sudoku(feat)
        
        true = labels[i].reshape((9,9))+1
        
        if(abs(true - pred).sum()==0):
            correct += 1
        
    print(correct/feats.shape[0])

def solve_sudoku(game):
    
    game = game.replace('\n', '')
    game = game.replace(' ', '')
    game = np.array([int(j) for j in game]).reshape((9,9,1))
    game = norm(game)
    game = inference_sudoku(game)
    return game

You can put in any game in the "game" string to solve it. Just copy new_game string in the game string and modify the desired zeros.

In [ ]:
new_game = '''
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
      '''

game = '''
          0 0 0 7 0 0 0 9 6
          0 0 3 0 6 9 1 7 8
          0 0 7 2 0 0 5 0 0
          0 7 5 0 0 0 0 0 0
          9 0 1 0 0 0 3 0 0
          0 0 0 0 0 0 0 0 0
          0 0 9 0 0 0 0 0 1
          3 1 8 0 2 0 4 0 7
          2 4 0 0 0 5 0 0 0
      '''

game = solve_sudoku(game)

print('solved puzzle:\n')
print(game)

In [ ]:
np.sum(game, axis=1)

## <span style="color:green;text-decoration:underline">Normal Approach #1</span>
As we can see that this model only achieved an accuracy of 84% on the training set and is not ideal for solving a sudoku. Neural nets are built to generalise and are not ideal for this problem. There are better approaches to achieve this using normal programming like the one given below which uses [backtracking](https://www.geeksforgeeks.org/sudoku-backtracking-7/) to solve the problem.

In [ ]:
def solve(bo):
    find = find_empty(bo)
    if not find:
        return True
    else:
        row, col = find

    for i in range(1,10):
        if valid(bo, i, (row, col)):
            bo[row][col] = i

            if solve(bo):
                return True

            bo[row][col] = 0

    return False


def valid(bo, num, pos):
    # Check row
    for i in range(len(bo[0])):
        if bo[pos[0]][i] == num and pos[1] != i:
            return False

    # Check column
    for i in range(len(bo)):
        if bo[i][pos[1]] == num and pos[0] != i:
            return False

    # Check box
    box_x = pos[1] // 3
    box_y = pos[0] // 3

    for i in range(box_y*3, box_y*3 + 3):
        for j in range(box_x * 3, box_x*3 + 3):
            if bo[i][j] == num and (i,j) != pos:
                return False

    return True


def print_board(bo):
    for i in range(len(bo)):
        if i % 3 == 0 and i != 0:
            print("- - - - - - - - - - - - - ")

        for j in range(len(bo[0])):
            if j % 3 == 0 and j != 0:
                print(" | ", end="")

            if j == 8:
                print(bo[i][j])
            else:
                print(str(bo[i][j]) + " ", end="")


def find_empty(bo):
    for i in range(len(bo)):
        for j in range(len(bo[0])):
            if bo[i][j] == 0:
                return (i, j)  # row, col

    return None

In [ ]:
%%time
game = '''
          0 0 0 7 0 0 0 9 6
          0 0 3 0 6 9 1 7 8
          0 0 7 2 0 0 5 0 0
          0 7 5 0 0 0 0 0 0
          9 0 1 0 0 0 3 0 0
          0 0 0 0 0 0 0 0 0
          0 0 9 0 0 0 0 0 1
          3 1 8 0 2 0 4 0 7
          2 4 0 0 0 5 0 0 0
      '''
game = game.strip().split("\n")
board = []
for i in game:
    t = i.replace(' ','').strip()
    t = list(t)
    t = list(map(int,t))
    board.append(t)
    
if solve(board):
    print_board(board)
else:
    print("Can't be solved.")

In [ ]:
np.sum(board, axis=1)

This works like a charm and it is also very fast.
<br>Let's check how fast is this algo. We will solve first 1000 problems and see the accuracy and speed!!

In [ ]:
val_set = data.iloc[:1000]


from tqdm import tqdm
quiz_list = list(val_set['quizzes'])
sol_list = list(val_set['solutions'])
val_quiz = []
val_sol = []
for i,j in tqdm(zip(quiz_list,sol_list)):
    q = np.array(list(map(int,list(i)))).reshape(9,9)
    s = np.array(list(map(int,list(j)))).reshape(9,9)
    val_quiz.append(q)
    val_sol.append(s)

In [ ]:
%%time
count = 0
for i,j in tqdm(zip(val_quiz,val_sol)):
    if solve(i):
        if (i==j).all():
            count+=1
    else:
        pass
    
print("{}/1000 solved!! That's {}% accuracy.\n".format(count,(count/1000.0)*100))

We can see that although it achieved 100% accuracy it is slow as it took a lot of time| to solve 1000 problems. Let's look at some better approaches.

## <span style="color:green;text-decoration:underline">Normal Approach #2</span>
A faster approach is to use the [**Naked Twin**](https://medium.com/@anandpathak69/solving-sudoku-using-naked-twin-strategies-f7ed23ea867f) approach which is a lot faster than backtracking.

In [ ]:
import numpy as np
import pandas as pd

import collections

rows = 'ABCDEFGHI'
cols = '123456789'

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [s + t for s in A for t in B]


boxes = cross(rows, cols)

row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789')]
unitlist = row_units + column_units + square_units 
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s], [])) - set([s])) for s in boxes)


def assign_value(values, box, value):
    """
    Please use this function to update your values dictionary!
    Assigns a value to a given box. If it updates the board record it.
    """
    values[box] = value
    return values


def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}
    Returns:
        the values dictionary with the naked twins eliminated from peers.
    """

    # Find all instances of naked twins
    for unit in unitlist:
        # Occurrences dict
        unit_values_counter = collections.Counter([values[box] for box in unit])
        for twins, count in unit_values_counter.items():
            # twins will occur twice in a unit, triples will occur three times, and quads four times
            if 1 < count == len(twins):
                for box in unit:
                    # for all boxes except twins boxes in a unit,
                    # remove all potential values that exist in twins, triples, quads..
                    if values[box] != twins and set(values[box]).intersection(set(twins)):
                        for digit in twins:
                            values = assign_value(values, box, values[box].replace(digit, ''))
    return values


def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Args:
        grid(string) - A grid in string form.
    Returns:
        A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value, then the value will be '123456789'.
    """
    chars = []
    digits = '123456789'
    for c in grid:
        if c in digits:
            chars.append(c)
        if c == '0':
            chars.append(digits)
    assert len(chars) == 81
    return dict(zip(boxes, chars))


def display(values):
    """
    Display the values as a 2-D grid.
    Args:
        values(dict): The sudoku in dictionary form
    """
    width = 1 + max(len(values[s]) for s in boxes)
    line = '+'.join(['-' * (width * 3)] * 3)
    for r in rows:
        print(''.join(values[r + c].center(width) + ('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    print


def eliminate(values):
    """
        Go through all the boxes, and whenever there is a box with a value, eliminate this value from the values of all its peers.
        Input: A sudoku in dictionary form.
        Output: The resulting sudoku in dictionary form.
        """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit, '')
    return values


def only_choice(values):
    """
        Go through all the units, and whenever there is a unit with a value that only fits in one box, assign the value to this box.
        Input: A sudoku in dictionary form.
        Output: The resulting sudoku in dictionary form.
        """
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values


def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    stalled = False
    while not stalled:
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        values = eliminate(values)
        values = only_choice(values)
        values = naked_twins(values)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        stalled = solved_values_before == solved_values_after
        if len([box for box in values.keys() if len(values[box]) == 0]):
            #display(values)
            return False
    return values


def search(values):
    "Using depth-first search and propagation, create a search tree and solve the sudoku."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False  ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes):
        return values  ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    min_possibility_box = min([box for box in boxes if len(values[box]) > 1])
    # Now use recursion to solve each one of the resulting sudokus, and if one returns a value (not False), return that answer!
    for digit in values[min_possibility_box]:
        new_sudoku = values.copy()
        new_sudoku[min_possibility_box] = digit
        attempt = search(new_sudoku)
        if attempt:
            return attempt


def solve2(grid):

    values = grid_values(grid)
    values = search(values)
    return values

In [ ]:
%%time
count = 0
for row in tqdm(data.head(1000).iterrows()):
    if (solve2(row[1]["quizzes"]) == grid_values(row[1]["solutions"])):
        count+=1
        
print("{}/1,000 solved!! That's {}% accuracy.\n".format(count,(count/1000.0)*100))

Thats really fast compared to backtracking. Let's improve it a little bit more using multithreding(using all cpu cores in parallel).

In [ ]:
%%time
from multiprocessing import Pool
num_partitions = 100 #number of partitions to split dataframe
num_cores = 6 #number of cores on your machine

def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    pool.map(func, df_split)
    pool.close()
    pool.join()

def solve_and_verify(data):
    for row in data.iterrows():
        assert solve2(row[1]["quizzes"]) == grid_values(row[1]["solutions"])
    
parallelize_dataframe(data.head(1000), solve_and_verify)

As you can see this is a lot faster(almost real-time)!!

## Conclusion
* From the above experiments to solve a sudoku, we found out that neural nets are not very accurate in this task and are not ideal candidates for this. They perform better in more generic tasks rather than very specific and arithmetic tasks.
* Using backtracking we can solve a sudoku but it will be very slow as it uses multiple combinations as it proceeds.
* The best approach by-far according to this experiment is Naked Twins approach. There are better versions of this approach out there that are even faster than this called Naked Triplets and even Naked Quadruples.

### TODO: 
1. Creating a image sudoku extractor and solver.
2. Write a detailed medium article.

**NOTE:**<span style="color:red"> If you like my work, or you want to use it please give an </span>***UPVOTE***. <span style="color:red">It really motivates me to create more and better tutorials.</span> 

## Credits
1. https://www.kaggle.com/azureq/sudoku-ai-first-100k-in-7-mins
2. https://towardsdatascience.com/solving-sudoku-with-convolution-neural-network-keras-655ba4be3b11
3. https://techwithtim.net/tutorials/python-programming/sudoku-solver-backtracking/